In [ ]:
exec('from __future__ import unicode_literals')

import os
import sys
import random

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../onmt'))
if module_path not in sys.path:
    sys.path.append(module_path)

from itertools import repeat

from onmt.utils.logging import init_logger
from onmt.utils.misc import split_corpus
from onmt.translate.translator import build_translator

import onmt.opts as opts
from onmt.utils.parse import ArgumentParser
from kp_gen_eval import _get_parser


from nltk.corpus import stopwords
stoplist = stopwords.words('english')

from string import punctuation
import onmt.keyphrase.pke as pke
from nltk.corpus import stopwords

import onmt.keyphrase.kp_inference as kp_inference

In [37]:
import importlib
importlib.reload(kp_inference)

<module 'onmt.keyphrase.kp_inference' from '/Users/memray/project/kp/OpenNMT-kpg/onmt/keyphrase/kp_inference.py'>

### Load a paragraph from 10-K docs (assume current directory is OpenNMT-kpg/notebook/)

In [4]:
data_root_path = '../data/salesforce/10k'
print(os.path.abspath(data_root_path))
doc_dicts = []
for subdir, dirs, files in os.walk(data_root_path):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".txt"):
#             print(filepath)
            text = open(filepath, 'r').readlines()
            pars = [l for l in text if len(l.split()) > 20]
            doc = {'name': file, 'path': filepath, 'text': text, 'paragraph': pars}
#             print('#line=%d, #par=%d' % (len(text), len(pars)))
            doc_dicts.append(doc)
    
print('Loaded #(10k docs)=%d' % (len(doc_dicts)))

/Users/memray/project/kp/OpenNMT-kpg/data/salesforce/10k
Loaded #(10k docs)=11252


##### Sample a paragraph

In [12]:
doc_id = random.randint(0, len(doc_dicts))
doc = doc_dicts[doc_id]
pars = doc['paragraph']
par_id = random.randint(0, len(doc_dicts[doc_id]['paragraph']))
text_to_extract = pars[par_id]
print(doc_id)
print(par_id)
print(text_to_extract)

7705
3
CSX was incorporated in 1978 under Virginia law. In 1980, the Company completed the merger of the Chessie System and Seaboard Coast Line Industries into CSX. The merger allowed the Company to connect northern population centers and Appalachian coal fields to growing southeastern markets. Later, the Company’s acquisition of key portions of Conrail, Inc. ("Conrail") allowed CSXT to link the northeast, including New England and the New York metropolitan area, with Chicago and midwestern markets as well as the growing areas in the Southeast already served by CSXT. This current rail network allows the Company to directly serve every major market in the eastern United States with safe, dependable, environmentally responsible and fuel efficient freight transportation and intermodal service.



In [11]:
# doc_id = 9539
# par_id = 67
doc = doc_dicts[doc_id]
pars = doc['paragraph']
par = pars[par_id]
print(doc_id)
print(par_id)
print(par)

IndexError: list index out of range

### Or load a PROD description

In [35]:
PROD_path = '../data/salesforce/SF_Prod'

prod_dicts = []
for subdir, dirs, files in os.walk(PROD_path):
    for file in files:
        filepath = subdir + os.sep + file
        text = open(filepath, 'r').readlines()
        text = '\n'.join([l.strip() for l in text])
        doc = {'name': file, 'path': filepath, 'text': text}
        prod_dicts.append(doc)

print('Loaded #(PROD docs)=%d' % (len(prod_dicts)))

doc_id = random.randint(0, len(prod_dicts))
doc = prod_dicts[doc_id]
text_to_extract = doc['text']
print(doc_id)
print(doc['name'])
print(text_to_extract)

Loaded #(PROD docs)=20
15
Sales_einstein
an artificial intelligence built into the core of the salesforce platform
it helps sales representatives win more deals with intelligent automation smarter scoring actionable insights and predictive forecasting
increases sales productivity at every step of the sales process with key predictions intelligent recommendations and timely automation
it is a data science department that learns from your teams sales activities and crm data and helps you identify the best leads convert opportunities more efficiently and retain customers with ease
it also includes the sales analytics app and inbox


### Supervised Deep Keyphrase Model

In [40]:
parser = _get_parser()
config_path = '../config/translate/config-rnn-keyphrase.yml'
one2one_ckpt_path = '../models/keyphrase/meng17-one2one-kp20k-topmodels/kp20k-meng17-one2one-rnn-BS128-LR0.05-Layer1-Dim150-Emb100-Dropout0.0-Copytrue-Covfalse-Contboth-IF1_step_30000.pt'
one2seq_ckpt_path = '../models/keyphrase/meng17-one2seq-kp20k-topmodels/kp20k-meng17-verbatim_append-rnn-BS64-LR0.05-Layer1-Dim150-Emb100-Dropout0.0-Copytrue-Reusetrue-Covtrue-PEfalse-Contboth-IF1_step_50000.pt'
opt = parser.parse_args('-config %s' % (config_path))
setattr(opt, 'models', [one2one_ckpt_path])

translator = build_translator(opt, report_score=False)

In [41]:
scores, predictions = translator.translate(
    src=[text_to_extract],
    tgt=None,
    src_dir=opt.src_dir,
    batch_size=opt.batch_size,
    attn_debug=opt.attn_debug,
    opt=opt
)

Translating 10/1


/Users/memray/Project/anaconda3/lib/python3.6/site-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)


RuntimeError: Expected object of scalar type Byte but got scalar type Bool for argument #3 'other' in call to _th_ior_

In [16]:
print('Paragraph:\n\t'+text_to_extract)
print('Top predictions:')
keyphrases = [kp.strip() for kp in predictions[0] if (not kp.lower().strip() in stoplist) and (kp != '<unk>' )]
for kp_id, kp in enumerate(keyphrases[: min(len(keyphrases), 20)]):
    print('\t%d: %s' % (kp_id+1, kp))

Paragraph:
	CSX was incorporated in 1978 under Virginia law. In 1980, the Company completed the merger of the Chessie System and Seaboard Coast Line Industries into CSX. The merger allowed the Company to connect northern population centers and Appalachian coal fields to growing southeastern markets. Later, the Company’s acquisition of key portions of Conrail, Inc. ("Conrail") allowed CSXT to link the northeast, including New England and the New York metropolitan area, with Chicago and midwestern markets as well as the growing areas in the Southeast already served by CSXT. This current rail network allows the Company to directly serve every major market in the eastern United States with safe, dependable, environmentally responsible and fuel efficient freight transportation and intermodal service.

Top predictions:
	1: coal fields
	2: CSX
	3: 1978
	4: Virginia
	5: Company
	6: Chessie
	7: merger
	8: New
	9: CSX.
	10: freight transportation
	11: law.
	12: rail network
	13: Seaboard
	14: co

### PKE models

#### TF-IDF

In [23]:
dataset_name = 'SF_Prod'
dataset_path = '../data/salesforce/%s/' % dataset_name
_ = kp_inference.extract_pke(text_to_extract, method='tfidf' , dataset_path=dataset_path,
            df_path=os.path.abspath(dataset_path + '../%s.df.tsv.gz' % dataset_name), top_k=20)

	1: salesforce (0.0000)
	2: salesforce built (0.0000)
	3: salesforce built intelligence (0.0000)
	4: built (0.0000)
	5: built intelligence (0.0000)
	6: built intelligence artificial (0.0000)
	7: intelligence (0.0000)
	8: intelligence artificial (0.0000)
	9: intelligence artificial core (0.0000)
	10: artificial (0.0000)
	11: artificial core (0.0000)
	12: artificial core platform (0.0000)
	13: core (0.0000)
	14: core platform (0.0000)
	15: platform (0.0000)
	16: predictive (0.0000)
	17: predictive forecasting (0.0000)
	18: predictive forecasting more (0.0000)
	19: forecasting (0.0000)
	20: forecasting more (0.0000)


#### YAKE

In [38]:
_ = kp_inference.extract_pke(text_to_extract, method='yake', top_k=20)

	1: artificial intelligence built (0.0009)
	2: smarter scoring actionable (0.0009)
	3: scoring actionable insights (0.0009)
	4: best leads convert (0.0009)
	5: leads convert opportunities (0.0009)
	6: automation smarter scoring (0.0011)
	7: key predictions intelligent (0.0011)
	8: predictions intelligent recommendations (0.0011)
	9: data science department (0.0011)
	10: increases sales productivity (0.0011)
	11: intelligent automation smarter (0.0012)
	12: sales representatives win (0.0014)
	13: teams sales activities (0.0014)
	14: sales analytics app (0.0014)
	15: helps sales representatives (0.0015)
	16: salesforce platform (0.0074)
	17: predictive forecasting (0.0074)
	18: artificial intelligence (0.0094)
	19: intelligence built (0.0094)
	20: representatives win (0.0094)


#### TextRank

In [24]:
# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor.load_document(input=text_to_extract,
                        language='en_core_web_sm',
                        normalization=None)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor.candidate_weighting(window=2,
                              pos=pos,
                              top_percent=0.33)

# 4. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)
for kp_id, kp in enumerate(keyphrases):
    print('\t%d: %s (%.4f)' % (kp_id+1, kp[0], kp[1]))

	1: intelligent automation timely (0.1305)
	2: automation intelligent insights (0.1305)
	3: artificial core (0.0776)
	4: data (0.0539)
	5: analytics (0.0434)
	6: predictions (0.0434)
	7: forecasting (0.0434)
	8: convert (0.0329)
